# __Predicting cell biological response__

In [62]:
import wandb
import pandas as pd
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
import logging

#### Run other .ipybn files

In [63]:
# %run NN_model.ipynb
from NN_model import *

%run NN_dataset.ipynb

         D1        D2    D3   D4        D5        D6        D7        D8  \
0  0.000000  0.497009  0.10  0.0  0.132956  0.678031  0.273166  0.585445   
1  0.366667  0.606291  0.05  0.0  0.111209  0.803455  0.106105  0.411754   
2  0.033300  0.480124  0.00  0.0  0.209791  0.610350  0.356453  0.517720   
3  0.000000  0.538825  0.00  0.5  0.196344  0.724230  0.235606  0.288764   
4  0.100000  0.517794  0.00  0.0  0.494734  0.781422  0.154361  0.303809   

         D9       D10  ...  D1768  D1769  D1770  D1771  D1772  D1773  D1774  \
0  0.743663  0.243144  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1  0.836582  0.106480  ...    1.0    1.0    1.0    0.0    1.0    0.0    0.0   
2  0.679051  0.352308  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3  0.805110  0.208989  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4  0.812646  0.125177  ...    0.0    0.0    0.0    0.0    0.0    0.0    0.0   

   D1775  D1776  target  
0    0.0    0.0       1  
1    1.0    0.0 

#### Dataset representation for NN model

In [64]:
class CellDataset(Dataset):
    def __init__(self, data: pd.DataFrame, normalize: bool):
        # Save predictors as DataFrame
        self.cell_descriptors = data.drop(columns=['target'])
        res = data['target']

        #if normalize:
            #res = res.apply(lambda x: -1 if x == 0 else 1)

        # Save target as DataFrame
        self.cell_response = res.astype('float64').to_frame()

    def __len__(self):
        return len(self.cell_descriptors)

    def __getitem__(self, idx):
        desc = self.cell_descriptors.iloc[idx]
        res = self.cell_response.iloc[idx]
        return desc.values, res.values
    
    def get_input_size(self):
        return self.cell_descriptors.shape[1]

## Wandb Project

In [65]:
wandb.login()

%env "WANDB_NOTEBOOK_NAME" "NN-z1"

run = wandb.init(
    entity = "matus13579",  #dont change
    project = "NN-z1",      #dont change
    name = "relu_test"       #run name
    #id =                   #define run with ID (used for resuming)
    #resume = True           #resume run
    )

#show graphs in Jupyter Notebook
#%%wandb                   

logger = logging.getLogger("wandb")
logger.setLevel(logging.ERROR)

env: "WANDB_NOTEBOOK_NAME"="NN-z1"


Problem at: c:\Users\matul\anaconda3\envs\NN\Lib\site-packages\wandb\sdk\wandb_init.py 854 getcaller


CommError: Run initialization has timed out after 90.0 sec. 
Please refer to the documentation for additional information: https://docs.wandb.ai/guides/track/tracking-faq#initstarterror-error-communicating-with-wandb-process-

## Experiments

In [ ]:
data = pd.read_csv('bioresponse.csv')

#### Load Dataset and Initialize NN model

In [ ]:
def setup_experiment(current_data):
    # Split dataset -> 20% testing, 80% training
    # Stratified split = each dataset has equal amounts of each class (saved in column 'target')
    train, test = train_test_split(current_data, test_size = 0.2, stratify = current_data['target']) 

    # Initlize dataset for NN
    train_data = CellDataset(train, False)
    test_data = CellDataset(test, False)

    # Create NN and training class
    mlp = MultiLayerPerceptron(train_data.get_input_size())
    trainer = Trainer(config, mlp)

    # Load dataset
    trainer.load_dataset(train_data, test_data)

    return trainer

### Training loop

In [ ]:
def run_experiment(myrun, trainer: Trainer, no_epochs):
    best_model = None
    best_accuracy = None

    for epoch in range(no_epochs):
        # Train model
        trainer.train()

        # Get metrics
        metrics = trainer.evaluate()

        loss_tr, loss_val = trainer.mean_loss()
        
        print (f"Epoch {epoch}")
        print (f"loss_training: {loss_tr} | loss_validate: {loss_val}")

        myrun.log({"loss_training": loss_tr})
        myrun.log({"loss_validate": loss_val})
        myrun.log({"accuracy": metrics.accuracy})
        myrun.log({"f1_score": metrics.accuracy})
        
        if (best_accuracy is None) or (best_accuracy > metrics.accuracy):
            best_accuracy = metrics.accuracy
            best_model = trainer.model


#### Experiment 1

In [ ]:
current_data = correlation_selection_merged(data)

config.activation_fn = "optimal"

In [ ]:
current_trainer = setup_experiment(current_data)
run_experiment(run, current_trainer, 100)

Using cpu device for training
Epoch 0
loss_training: 9.517757415771484 | loss_validate: 9.505755424499512
Epoch 1
loss_training: 9.532607078552246 | loss_validate: 9.5322265625
Epoch 2
loss_training: 9.556146621704102 | loss_validate: 9.48403549194336
Epoch 3
loss_training: 9.536160469055176 | loss_validate: 9.522931098937988
Epoch 4
loss_training: 9.533585548400879 | loss_validate: 9.49963665008545
Epoch 5
loss_training: 9.496227264404297 | loss_validate: 9.517797470092773
Epoch 6
loss_training: 9.606687545776367 | loss_validate: 9.498635292053223
Epoch 7
loss_training: 9.499774932861328 | loss_validate: 9.525062561035156
Epoch 8
loss_training: 9.497215270996094 | loss_validate: 9.485235214233398
Epoch 9
loss_training: 9.5120267868042 | loss_validate: 9.519922256469727
Epoch 10
loss_training: 9.439984321594238 | loss_validate: 9.492541313171387
Epoch 11
loss_training: 9.454813957214355 | loss_validate: 9.498263359069824
Epoch 12
loss_training: 9.513028144836426 | loss_validate: 9.4832

KeyboardInterrupt: 